In [1]:
import numpy as np
from ctapipe.io import EventSource
from ctapipe.io import EventSeeker
import matplotlib.pyplot as plt
import numpy as np
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['font.size'] = 20


In [2]:
source = EventSource(input_url="/Users/thomasvuillaume/Work/CTA/Data/LST1/LST-1.1.Run00088.0000.fits.fz",
                      max_events=20)

Not an HDF5 file


In [3]:
def format_axes(ax):
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.tick_params(
    axis='x',       # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
    ax.tick_params(
    axis='y',       # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off
    
    return ax

In [ ]:
geom = CameraGeometry.from_name("LSTCam-002")
evt_id = 203 # The id of the selected event

for i, ev in enumerate(source):
    N_modules = 7*265
    print(ev.lst.tel[0].evt.event_id)

    #if((ev.lst.tel[0].evt.event_id<19800) or (ev.lst.tel[0].evt.event_id>19860)):
    #    continue

    std_signal = np.zeros(1855)
    for pixel in range(0, N_modules):
        std_signal[pixel] = np.max(ev.r0.tel[0].waveform[0, pixel, 2:38])

    if(np.size(std_signal[std_signal>1000.]) < 15):
        continue  
    print(f"Event {ev.lst.tel[0].evt.event_id}, Max: {np.max(std_signal)} counts")


    #geom = CameraGeometry.from_name("LSTCam-002")
    fig, ax = plt.subplots(figsize=(10,10))

    #disp0 = CameraDisplay(geom, ax=ax)
    disp0 = CameraDisplay(ev.inst.subarray.tels[0].camera, ax=ax)
    disp0.cmap = 'viridis'
    disp0.image = std_signal
    disp0.add_colorbar(ax=ax)

    # Establish max and min
    sort = np.argsort(std_signal)
    min_color = std_signal[sort][7] # There was one cluster off
    max_color = std_signal[sort][-2]
    max_color = np.max(std_signal)

    disp0.set_limits_minmax(min_color, max_color)
    ax.set_title(f"Event {ev.lst.tel[0].evt.event_id}")
    format_axes(ax)
#     fig.savefig("Images_LST/Event_%i.png"%(ev.lst.tel[0].evt.event_id))

    plt.show()
    if(ev.lst.tel[0].evt.event_id==evt_id):
        break

In [ ]:
# If you want to make a movie with all the slices

max_color = np.max(std_signal)
sort = np.argsort(std_signal)
min_color = std_signal[sort][7]

for cell in range(1,39):
    print("cell",cell)
    fig, ax = plt.subplots(figsize=(10,10))
    disp0 = CameraDisplay(geom, ax=ax)
    disp0.cmap = 'viridis'
    disp0.add_colorbar(ax=ax)
    disp0.image = ev.r0.tel[0].waveform[0,:,cell]
    disp0.set_limits_minmax(min_color, max_color)
    format_axes(ax)
    ax.set_title(f"Event {ev.lst.tel[0].evt.event_id}, Time {cell} ns")
#     fig.savefig("Images_LST/for_gifs/Event_{:02d}_cell{:02d}.png".format(ev.lst.tel[0].evt.event_id,cell))
    #plt.show()